In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import torch.optim as optim

import torchmetrics

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"Device: {device}")

In [ ]:
batch_size = 64

train_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(0.5),
     transforms.RandomVerticalFlip(0.5),
     transforms.RandomRotation(24),
     transforms.ToTensor(),
     transforms.RandomErasing(0.4),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
dataset = ImageFolder("data")

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_dataset.dataset = ImageFolder("data", transform=train_transform)
val_dataset.dataset = ImageFolder("data", transform=val_test_transform)
test_dataset.dataset = ImageFolder("data", transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Number of images in training dataset: {len(train_dataset)}")
print(f"Number of images in validation dataset: {len(val_dataset)}")
print(f"Number of images in test dataset: {len(test_dataset)}")

In [ ]:
plt.figure(figsize = (20,10))

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# get some random training images
images, labels = next(iter(train_loader))

# show images
imshow(torchvision.utils.make_grid(images))

print(list(dataset.classes[labels[j]] for j in range(batch_size)))

In [ ]:
images[0:1].size()

In [32]:
results = pd.DataFrame(columns=['model', 'max_val_acc'])

In [24]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2)
        # self.conv3 = nn.Conv2d(16, 32, 5)
        # self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        # x = self.pool3(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
def evaluate(model, dataloader, device, num_classes):
    model.eval()
    metric_acc = torchmetrics.classification.Accuracy(task='multiclass', num_classes=num_classes).to(device)

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)

        with torch.inference_mode():
            logits = model(X)
            _, preds = torch.max(logits, dim=1)

            metric_acc(preds, y)

    return metric_acc.compute()

In [22]:
def train(model, criterion, optimizer, train_dataloader, val_dataloader, device, num_classes, epochs):
    best_val_acc = 0.0
    
    metric_acc = torchmetrics.classification.Accuracy(task='multiclass', num_classes=num_classes).to(device)
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            metric_acc(outputs, labels)

            # print statistics
            running_loss += loss.item()

        loss = running_loss / 2000
        train_acc = metric_acc.compute()
        val_acc = evaluate(model, val_dataloader, device, num_classes)
        best_val_acc = max(best_val_acc, val_acc)
        print(f'[{epoch+1}/{epochs}] {loss=:.4f}, {train_acc=:.4f}, {val_acc=:.4f}')

        metric_acc.reset()

    print('Finished Training')
    return best_val_acc

In [ ]:

classifier = ImageClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

max_val_acc = train(classifier, criterion, optimizer, train_loader, val_loader, device, len(dataset.classes), epochs=1)

results = pd.concat([results, pd.DataFrame({'model': [str(classifier)], 'max_val_acc': [max_val_acc]})], ignore_index=True)

In [ ]:
test_accuracy = evaluate(classifier, test_loader, device, len(dataset.classes)).item()
print(f'{test_accuracy=:.4f}')